In [1]:
# Import required libraries
import pandas as pd
from typing import List, Dict, Optional
import csv
from datetime import datetime

In [4]:
class RecipeManager:
    """
    A class to manage recipes in a CSV-based system.
    """
    
    def __init__(self, csv_file: str):
        """
        Goal: Initialize the RecipeManager with a CSV file
        Parameters:
            csv_file (str): Path to the CSV file containing recipes
        Output: None
        """
        self.csv_file = csv_file
        self.recipes_df = pd.read_csv(csv_file)
        
    def add_recipe(self, recipe_data: Dict) -> bool:
        """
        Goal: Add a new recipe to the system
        Parameters:
            recipe_data (Dict): Dictionary containing recipe information with keys:
                              'Recipe Name', 'Ingredients', 'Steps', 'Cuisine', 'Preparation Time'
        Output:
            bool: True if recipe added successfully, False otherwise
        """
        try:
            # Generate new Recipe ID
            max_id = int(self.recipes_df['Recipe ID'].str.extract('(\d+)').astype(int).max())
            new_id = f'R{str(max_id + 1).zfill(3)}'
            recipe_data['Recipe ID'] = new_id
            
            # Append new recipe
            self.recipes_df = pd.concat([self.recipes_df, pd.DataFrame([recipe_data])], ignore_index=True)
            self.save_changes()
            return True
        except Exception as e:
            print(f"Error adding recipe: {e}")
            return False

    def update_recipe(self, recipe_id: str, updated_data: Dict) -> bool:
        """
        Goal: Update an existing recipe's information
        Parameters:
            recipe_id (str): The ID of the recipe to update
            updated_data (Dict): Dictionary containing the updated recipe information
        Output:
            bool: True if recipe updated successfully, False otherwise
        """
        try:
            if recipe_id in self.recipes_df['Recipe ID'].values:
                idx = self.recipes_df[self.recipes_df['Recipe ID'] == recipe_id].index[0]
                for key, value in updated_data.items():
                    if key in self.recipes_df.columns:
                        self.recipes_df.at[idx, key] = value
                self.save_changes()
                return True
            return False
        except Exception as e:
            print(f"Error updating recipe: {e}")
            return False

    def delete_recipe(self, recipe_id: str) -> bool:
        """
        Goal: Delete a recipe from the system
        Parameters:
            recipe_id (str): The ID of the recipe to delete
        Output:
            bool: True if recipe deleted successfully, False otherwise
        """
        try:
            if recipe_id in self.recipes_df['Recipe ID'].values:
                self.recipes_df = self.recipes_df[self.recipes_df['Recipe ID'] != recipe_id]
                self.save_changes()
                return True
            return False
        except Exception as e:
            print(f"Error deleting recipe: {e}")
            return False

    def search_by_name(self, name: str) -> pd.DataFrame:
        """
        Goal: Search for recipes by name
        Parameters:
            name (str): Name or partial name to search for
        Output:
            pd.DataFrame: DataFrame containing matching recipes
        """
        return self.recipes_df[self.recipes_df['Recipe Name'].str.contains(name, case=False, na=False)]

    def search_by_cuisine(self, cuisine: str) -> pd.DataFrame:
        """
        Goal: Search for recipes by cuisine type
        Parameters:
            cuisine (str): Cuisine type to search for
        Output:
            pd.DataFrame: DataFrame containing matching recipes
        """
        return self.recipes_df[self.recipes_df['Cuisine'].str.contains(cuisine, case=False, na=False)]

    def search_by_ingredient(self, ingredient: str) -> pd.DataFrame:
        """
        Goal: Search for recipes containing a specific ingredient
        Parameters:
            ingredient (str): Ingredient to search for
        Output:
            pd.DataFrame: DataFrame containing matching recipes
        """
        return self.recipes_df[self.recipes_df['Ingredients'].str.contains(ingredient, case=False, na=False)]

    def filter_by_time(self, max_time: int) -> pd.DataFrame:
        """
        Goal: Filter recipes by maximum preparation time
        Parameters:
            max_time (int): Maximum preparation time in minutes
        Output:
            pd.DataFrame: DataFrame containing recipes within time limit
        """
        return self.recipes_df[self.recipes_df['Preparation Time'].astype(int) <= max_time]

    def get_unique_ingredients(self) -> List[str]:
        """
        Goal: Get a list of all unique ingredients across all recipes
        Parameters: None
        Output:
            List[str]: Sorted list of unique ingredients
        """
        all_ingredients = []
        for ingredients in self.recipes_df['Ingredients']:
            ingredients_list = [i.strip() for i in ingredients.split(',')]
            all_ingredients.extend(ingredients_list)
        return sorted(list(set(all_ingredients)))

    def generate_shopping_list(self, recipe_ids: List[str], available_ingredients: List[str] = None) -> List[str]:
        """
        Goal: Generate a shopping list for selected recipes
        Parameters:
            recipe_ids (List[str]): List of recipe IDs to include
            available_ingredients (List[str], optional): List of ingredients already available
        Output:
            List[str]: Sorted list of required ingredients
        """
        needed_ingredients = set()
        for recipe_id in recipe_ids:
            recipe = self.recipes_df[self.recipes_df['Recipe ID'] == recipe_id]
            if not recipe.empty:
                ingredients = recipe['Ingredients'].iloc[0].split(',')
                needed_ingredients.update([i.strip() for i in ingredients])
        
        if available_ingredients:
            available_ingredients = set([i.strip() for i in available_ingredients])
            needed_ingredients = needed_ingredients - available_ingredients
            
        return sorted(list(needed_ingredients))

    def save_changes(self):
        """
        Goal: Save current recipe data to CSV file
        Parameters: None
        Output: None
        """
        self.recipes_df.to_csv(self.csv_file, index=False)


<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
C:\Users\rafiq\AppData\Local\Temp\ipykernel_16788\3862032416.py:11: SyntaxWarning: invalid escape sequence '\d'
  max_id = int(self.recipes_df['Recipe ID'].str.extract('(\d+)').astype(int).max())


In [6]:
def display_menu():
    """
    Goal: Display the main menu options and get user choice
    Parameters: None
    Output:
        str: User's menu choice (1-8)
    """
    print("\n=== Recipe Management System ===")
    print("1. Add New Recipe")
    print("2. Update Recipe")
    print("3. Delete Recipe")
    print("4. Search Recipes")
    print("5. Generate Shopping List")
    print("6. View All Recipes")
    print("7. View Unique Ingredients")
    print("8. Exit")
    return input("Enter your choice (1-8): ")

def get_recipe_input() -> Dict:
    """
    Goal: Get recipe information from user input
    Parameters: None
    Output:
        Dict: Dictionary containing recipe information with keys:
             'Recipe Name', 'Ingredients', 'Steps', 'Cuisine', 'Preparation Time'
    """
    return {
        'Recipe Name': input("Enter recipe name: "),
        'Ingredients': input("Enter ingredients (comma-separated): "),
        'Steps': input("Enter preparation steps (comma-separated): "),
        'Cuisine': input("Enter cuisine type: "),
        'Preparation Time': int(input("Enter preparation time (minutes): "))
    }


In [10]:
def main():
    """
    Goal: Main function to run the Recipe Management System
    Parameters: None
    Output: None
    """
    recipe_manager = RecipeManager('recipe.csv')
    
    while True:
        choice = display_menu()
        
        if choice == '1':  # Add New Recipe
            recipe_data = get_recipe_input()
            if recipe_manager.add_recipe(recipe_data):
                print("Recipe added successfully!")
            else:
                print("Failed to add recipe.")

        elif choice == '2':  # Update Recipe
            recipe_id = input("Enter Recipe ID to update: ")
            recipe_data = get_recipe_input()
            if recipe_manager.update_recipe(recipe_id, recipe_data):
                print("Recipe updated successfully!")
            else:
                print("Failed to update recipe.")

        elif choice == '3':  # Delete Recipe
            recipe_id = input("Enter Recipe ID to delete: ")
            if recipe_manager.delete_recipe(recipe_id):
                print("Recipe deleted successfully!")
            else:
                print("Failed to delete recipe.")

        elif choice == '4':  # Search Recipes
            print("\nSearch by:")
            print("1. Name")
            print("2. Cuisine")
            print("3. Ingredient")
            print("4. Preparation Time")
            search_choice = input("Enter choice (1-4): ")
            
            if search_choice == '1':
                name = input("Enter recipe name to search: ")
                results = recipe_manager.search_by_name(name)
            elif search_choice == '2':
                cuisine = input("Enter cuisine to search: ")
                results = recipe_manager.search_by_cuisine(cuisine)
            elif search_choice == '3':
                ingredient = input("Enter ingredient to search: ")
                results = recipe_manager.search_by_ingredient(ingredient)
            elif search_choice == '4':
                max_time = int(input("Enter maximum preparation time: "))
                results = recipe_manager.filter_by_time(max_time)
            
            if not results.empty:
                print("\nSearch Results:")
                print(results[['Recipe ID', 'Recipe Name', 'Cuisine', 'Preparation Time']])
            else:
                print("No recipes found.")

        elif choice == '5':  # Generate Shopping List
            recipe_ids = input("Enter Recipe IDs (comma-separated): ").split(',')
            available = input("Enter available ingredients (comma-separated, or press Enter to skip): ")
            available_ingredients = available.split(',') if available else None
            
            shopping_list = recipe_manager.generate_shopping_list(recipe_ids, available_ingredients)
            print("\nShopping List:")
            for item in shopping_list:
                print(f"- {item}")

        elif choice == '6':  # View All Recipes
            print("\nAll Recipes:")
            print(recipe_manager.recipes_df[['Recipe ID', 'Recipe Name', 'Cuisine', 'Preparation Time']])

        elif choice == '7':  # View Unique Ingredients
            print("\nUnique Ingredients:")
            for ingredient in recipe_manager.get_unique_ingredients():
                print(f"- {ingredient}")

        elif choice == '8':  # Exit
            print("Thank you for using Recipe Management System!")
            break

        else:
            print("Invalid choice. Please try again.")

In [14]:
main()


=== Recipe Management System ===
1. Add New Recipe
2. Update Recipe
3. Delete Recipe
4. Search Recipes
5. Generate Shopping List
6. View All Recipes
7. View Unique Ingredients
8. Exit

Search by:
1. Name
2. Cuisine
3. Ingredient
4. Preparation Time

Search Results:
   Recipe ID          Recipe Name         Cuisine  Preparation Time
0       R001  Spaghetti Carbonara         Italian                25
2       R003           Fried Rice         Chinese                30
10      R011            Shakshuka  Middle Eastern                30
14      R015         Banana Bread        American                60
16      R017             Pad Thai            Thai                40
33      R034       Coconut Shrimp        American                30
39      R040               Quiche          French                50
40      R041            Cornbread        American                30
47      R048             Moussaka           Greek                90

=== Recipe Management System ===
1. Add New Recipe
2